In [10]:
import pickle

import numpy as np
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [11]:
from sklearn.pipeline import make_pipeline

In [12]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1761304988993, experiment_id='1', last_update_time=1761304988993, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [13]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [14]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [15]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

2025/10/24 11:32:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/10/24 11:32:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run skittish-cod-947 at: http://127.0.0.1:5000/#/experiments/1/runs/3a634354861f454dacf1630e501ed39a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [17]:
from mlflow.tracking import MlflowClient


In [18]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '3a634354861f454dacf1630e501ed39a'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [25]:
# Load the logged model directly from the run and inspect its pipeline to find the vectorizer
import mlflow
from sklearn.feature_extraction import DictVectorizer

model_uri = f"runs:/{RUN_ID}/model"
print('model_uri =', model_uri)
loaded_model = mlflow.sklearn.load_model(model_uri)
print('loaded model type:', type(loaded_model))

if hasattr(loaded_model, 'named_steps'):
    print('pipeline steps:', loaded_model.named_steps.keys())
    dv = None
    for name, step in loaded_model.named_steps.items():
        if isinstance(step, DictVectorizer):
            dv = step
            print('Found DictVectorizer at step', name)
    if dv is None:
        print('DictVectorizer not found in pipeline')
else:
    print('Loaded object has no named_steps; it may not be a pipeline')

model_uri = runs:/3a634354861f454dacf1630e501ed39a/model


loaded model type: <class 'sklearn.pipeline.Pipeline'>
pipeline steps: dict_keys(['dictvectorizer', 'randomforestregressor'])
Found DictVectorizer at step dictvectorizer


In [26]:
# Extract the DictVectorizer from the loaded pipeline, save locally and (optionally) upload as a run artifact
import os, pickle

dv = loaded_model.named_steps.get('dictvectorizer')
if dv is None:
    # try lowercase/uppercase variations
    for k in loaded_model.named_steps:
        if 'dict' in k.lower():
            dv = loaded_model.named_steps[k]
            break

if dv is None:
    print('DictVectorizer not found in pipeline; cannot save')
else:
    out_path = 'dict_vectorizer.bin'
    with open(out_path, 'wb') as f:
        pickle.dump(dv, f)
    print('Saved DictVectorizer to', os.path.abspath(out_path))

    # upload to the same run under 'preprocessor' artifact path
    try:
        client.log_artifact(RUN_ID, out_path, artifact_path='preprocessor')
        print('Uploaded', out_path, 'to run', RUN_ID, 'under artifact path preprocessor')
    except Exception as e:
        print('Failed to upload artifact to mlflow run:', e)


Saved DictVectorizer to /workspaces/mlops/04-deployment/web-servies-mlflow/dict_vectorizer.bin
Uploaded dict_vectorizer.bin to run 3a634354861f454dacf1630e501ed39a under artifact path preprocessor


In [28]:
dv

DictVectorizer()